In [ ]:
import wave, os, glob
import noisereduce as nr
from scipy.io import wavfile
import matplotlib.pyplot as plt
import librosa.display
import sys
import numpy 
import librosa
numpy.set_printoptions(threshold=sys.maxsize)
from pydub import AudioSegment,silence
import math
from scipy.stats import variation 
import statistics
import pandas as pd
import itertools
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
folder_with_code_and_audio_files = # '..../Preprocessed_audio_files/AWS'

In [ ]:
pause_time_threshold = 50
speech_time_threshold = 100
segment_size_t = 1 # smaller segments to calculate mean within longer segment_size_t_windows
segment_size_t_window = 3 # segment size in seconds

In [ ]:
os.chdir(folder_with_code_and_audio_files)

In [ ]:
%run -i  "..../Stat_csv_files/ATAS_functions.ipynb"  #Change path as required

In [ ]:
# Get the csv file with the file names and the trimming intervals 
df_times = pd.read_csv("AWS_input.csv") # Change path if required


# This csv file is saved in the same location as the pre-processed audio files being used for the analysis
# This is a csv file with the exact file names under column name 'File_name'
# The second column with name 'Start_time'
# The third (optional) column with name 'End_time' - exclude this column if you want to consider the entire file for analysis

print (df_times)
df_times_dim = df_times.shape

   File_Name  Start_time 
0   24fa.wav          0.0
1   24ma.wav          0.0
2   24mb.wav          0.0
3    26f.wav          0.0
4   29ma.wav          0.0
5   29mb.wav          0.0
6   29mc.wav          0.0
7    32m.wav          0.0
8    34m.wav          0.0
9   35mb.wav          0.0
10  46ma.wav          0.0
11  50fa.wav          0.0
12  50fb.wav          0.0
13   57m.wav          0.0
14   60m.wav          0.0
15   62f.wav          0.0
16   62m.wav          0.0


In [ ]:
# this will take you to the folder you saved your files in and select just the wav files 
import glob, os
os.chdir(folder_with_code_and_audio_files)
all_files = []
for file in glob.glob("*.wav"):
    all_files.append(file)

In [ ]:

# This one will consider only the files that have names from the csv files that stores that audio file names for the files that need to be analyzed
df_together = []
for file_number in range(df_times_dim[0]):
    print(f"Analyzing file number: {file_number}")
    file_name = df_times.iloc[file_number,0]
    start_time = df_times.iloc[file_number,1]
    for f_name_current in all_files:
        if f_name_current == file_name:
            f_name_check = f_name_current
            print(f"Currently analyzing file: {f_name_current}")
            if df_times.shape[1] > 2:
            # Option 1: Use the end_time from the DataFrame
                end_time = df_times.iloc[file_number, 2]
            else:
            # Option 2: Use librosa to get the duration of the file
                end_time = librosa.get_duration(filename=f_name_current)
            print(end_time)
            x, sr = load_display_file_wf(f_name_current)
            x_nr, sr_nr, wav_file_nr = reduce_file_noise_wf (f_name_current,x,sr)
            x_nr_ex, sr_nr_ex, trim_file, f_trim_file,trim_file_viz, f_name_2 = wav_file_trim_wf (wav_file_nr, start_time, end_time)
            all_seg_details_silence_p_file, all_seg_details_silence_rem_file, seg_time_leng = event_detection_seg_wf_window(x_nr_ex, sr_nr_ex, segment_size_t_window, trim_file)            
            print(seg_time_leng)
            silence_p_final_end, silence_rem_final_end,ddf,long_p,short_p,long_p_durations,short_p_durations = detect_proper_events(all_seg_details_silence_p_file,all_seg_details_silence_rem_file,speech_time_threshold,pause_time_threshold, f_name_current, seg_time_leng)
            
            print(f"No. of long pauses: {len(long_p)}")
            print(f"Long pause durations: {long_p_durations}")
            print(f"No. of short pauses: {len(short_p)}")
            print(f"Short pause durations: {short_p_durations}")

            df,ddf = event_statistics_wf(f_name_current, f_trim_file, x_nr_ex, sr_nr_ex,silence_p_final_end, silence_rem_final_end,speech_time_threshold,pause_time_threshold, ddf, long_p, short_p)
            pp_t = os.getcwd()
            f_name_1 = f_name_current.rsplit('.', maxsplit=1)[0]
            ddf_csv_file = pp_t + '/' + f_name_1+ '_f.csv'
            ddf.to_csv(ddf_csv_file) 
            df_list = df.values.tolist()
            print(df_list)
            df_together.append(df_list[0])
            
df_together_csv = pd.DataFrame(df_together, columns= ['File_Name',
                              'Speech Time Threshold_ms',
                              'Pause Time Threshold_ms',
                              'Percent_Pause',
                              'Percent_Speech',
                              'Total_Duration_Unclipped_s',
                              'Total_Duration_Clipped_s',
                              'Speech_Duration_s', 
                              'Pause_Duration_s', 
                              'Speech_Events', 
                              'Pause_Events',
                              'Mean Speech_s',
                              'Std Dev Speech',
                              'CV Speech',
                              'Mean Pause_s',
                              'Std Dev Pause',
                              'CV Pause'                                                                                                       
                             ])
pp_t = os.getcwd()
file_tog_name = 'AWS_All_files_together_50_ms_1_win_50_150'
csv_file = pp_t + '/' + file_tog_name+ '_csv.csv'
df_together_csv.to_csv (csv_file, index = False, header=True)

In [ ]:
## Description of columns in the file AWS_All_files_together_50_ms_1_win_50_150.csv

# 'File_Name' - name of the file e.g. '24fa.wav'
# 'Speech Time Threshold_ms' - minimum speech event selection threshold (temporal) 
# 'Pause Time Threshold_ms' - minimum pause event selection threshold (temporal)
# 'Percent_Pause' - percent of (total duration of all pause events in file/ total duration of file)
# 'Percent_Speech' - percent of (total duration of all speech events in file/ total duration of file)
# 'Total_Duration_Unclipped_s' - Total original file duration
# 'Total_Duration_Clipped_s' - Total final file duration (for analysis)
# 'Speech_Duration_s' - Total duration of all speech events in file
# 'Pause_Duration_s'- Total duration of all pause events in file
# 'Speech_Events' - Speech events total count
# 'Pause_Events' - Pause events total counts
# 'Mean Speech_s' - Mean duration metric of all the speech event durations
# 'Std Dev Speech' - Standard deviation metric of all the speech event durations
# 'CV Speech' - Covariate of variation metric of all the speech event durations 
# 'Mean Pause_s' - Mean duration metric of all the pause event durations
# 'Std Dev Pause' - Standard deviation metric of all the pause event durations
# 'CV Pause'- Covariate of variation metric of all the pause event durations 


In [ ]:
# Description of columns in individual participant time series data csv file
# e.g. '24fa_f.csv'

# 'level_0' - This column holds previous event positions that were part of a 
#           multi-level index from an earlier stage in the function.

# 'index' - This column represents the original row position of each event in the DataFrame, 
#           which can be useful for backtracking or verifying the original sequence of detected events

# 'Time_start' - Start time of the detected event (in seconds, with respect to the time of the file)
# 'Time_end' - End time of the detected event (in seconds, with respect to the time of the file)
# 'Labels' - Type of event
# 'Lables_X' - Type of preceeding event
# 'Time_diff' - Duration of the detected event (in seconds)

